# Assistants API

Assistants API를 사용하면 자체 애플리케이션 내에 AI 도우미를 구축할 수 있습니다. 어시스턴트에는 instruction(지침)이 있으며 모델, 도구 및 파일을 활용하여 사용자 쿼리에 응답할 수 있습니다. Assistants API는 현재 코드 해석기, 파일 검색 및 함수 호출의 세 가지 유형의 도구를 지원합니다.  

Assistants 플레이그라운드(https://platform.openai.com/playground?mode=assistant)를 사용 하거나 아래에 설명된 것과 같이 Assistants API의 기능을 탐색할 수 있습니다.

### 코드 해석기 (Code Interpreter)

Assistants가 샌드박스 실행 환경에서 Python 코드를 작성하고 실행할 수 있습니다. 이 도구는 다양한 데이터와 포맷의 파일을 처리할 수 있으며, 데이터와 그래프 이미지가 포함된 파일을 생성할 수 있습니다. 코드 해석기를 사용하면 어시스턴트가 코드를 반복적으로 실행하여 어려운 코드 및 수학 문제를 해결할 수 있습니다. 어시스턴트가 실행에 실패한 코드를 작성하면 코드 실행이 성공할 때까지 다른 코드 실행을 시도하여 이 코드를 반복할 수 있습니다.

### Step 1 : assistant 생성  

- assistant는 model , instructions및 같은 여러 매개변수를 사용하여 사용자의 메시지에 응답하도록 구성할 수 있는 엔터티를 나타냅니다.

Instruction: 어시스턴트와 모델이 어떻게 행동하거나 반응해야 하는지 지시  
Model: 모델 지정   
Tools: API는 OpenAI에서 구축하고 호스팅하는 코드 해석기 및 검색을 지원합니다.  
Functions: 함수 호출을 사용하면 Assistants API에 함수를 설명하고 인수와 함께 호출해야 하는 함수를 지능적으로 반환하도록 할 수 있습니다.  

### 이 예에서는 코드 해석기 도구가 활성화된 개인 수학 교사인 도우미를 만듭니다.  
예) code interpreter가 활성화된 개인 수학 교사 assistant를 생성하여 주어진 수학 문제를 푸는 python code 생성

In [ ]:
# assistant 생성

### Step 2 : Thread 생성

스레드는 사용자와 하나 이상의 Assistant 간의 대화를 나타냅니다. 사용자(또는 AI 애플리케이션)가 어시스턴트와 대화를 시작할 때 스레드를 생성할 수 있습니다.  
사용자가 대화를 시작하자마자 사용자당 하나의 스레드를 생성하는 것이 좋습니다. 메시지를 생성하여 이 스레드에 사용자별 컨텍스트와 파일을 전달합니다.    

In [ ]:
# thread 생성

### Step 3 : Thread에 message 추가

사용자나 애플리케이션이 생성한 메시지의 내용은 스레드에 메시지 개체로 추가됩니다. 메시지에는 텍스트와 파일이 모두 포함될 수 있습니다. 스레드에 추가할 수 있는 메시지 수에는 제한이 없습니다. 모델의 컨텍스트 창에 맞지 ​​않는 모든 컨텍스트를 스마트하게 자릅니다. 어시스턴트 API를 사용하면 특정 RUN에 대해 모델에 전달되는 입력 토큰 수에 대한 제어를 위임합니다. 즉, 경우에 따라 어시스턴트 실행 비용에 대한 제어 권한이 줄어들지만 복잡성을 처리할 필요는 없습니다. 

In [ ]:
# thread에 message 추가
    #content="I need to solve the equation `3x + 11 = 14`. Can you help me?"

### Step 4 : Run 생성

모든 사용자 메시지가 스레드에 추가되면 특정 Assistant를 사용하여 스레드를 Run할 수 있습니다. Run을 생성하면 Assistant와 관련된 모델 및 도구를 사용하여 응답을 생성합니다. 이러한 응답은 스레드에 assistant메시지로 추가됩니다.

| 방식                          | 응답 속도       | 구현 난이도 | 사용 시점                   | 특징                              | 선택 기준 |
|------------------------------|--------------|----------|--------------------------|----------------------------------|------------------------------|
| **Streaming (`stream()`)**   | 빠름 (즉시 응답) | 복잡함     | 실시간 응답이 필요한 경우      | 이벤트 기반 응답, 자연스러운 대화 | ✅ 대화형 챗봇처럼 실시간 응답이 중요한 경우  |
|                              |              |          |                          |                                  | ✅ 결과를 빠르게 부분적으로 보여줘야 하는 경우 |
|                              |              |          |                          |                                  | ✅ API 요청이 오래 걸리는 작업을 실행하는 경우 (예: 코드 실행, 데이터 분석 등) |
| **Polling (`create_and_poll()`)** | 느림 (완료 후 응답) | 간단함     | 전체 결과가 필요할 때          | 한 번에 모든 결과를 가져옴       | ✅ 응답을 한 번에 처리해야 하는 경우 (예: 로그 분석, 문서 요약 등) |
|                              |              |          |                          |                                  | ✅ 실시간 응답이 필요하지 않은 백그라운드 작업 |
|                              |              |          |                          |                                  | ✅ 간단한 API 호출이 필요한 경우 |
 |


### 방법 1 - streaming 방식
OpenAI의 AssistantEventHandler를 사용하여 Run을 만들고 응답을 스트리밍할 수 있습니다.

In [ ]:
# 먼저, 이벤트 핸들러 클래스(EventHandler)를 생성하여
# 응답 스트림에서 이벤트를 어떻게 처리할지 정의합니다.
class EventHandler(AssistantEventHandler):    
  # 텍스트 생성이 완료되었을 때 호출되는 메서드
  def on_text_created(self, text) -> None:
  # 텍스트 생성 중간에 호출되는 메서드. 점진적으로 delta.value 출력
  def on_text_delta(self, delta, snapshot):
  # 도구 호출이 생성되었을 때 호출되는 메서드
  def on_tool_call_created(self, tool_call):
  # 도구 호출 중간에 호출되는 메서드
  def on_tool_call_delta(self, delta, snapshot):
      # 파이썬 코드
      # 파이썬 코드 실행 결과
          # 실행된 코드의 출력을 로그 형태로 출력

In [ ]:
# 그런 다음, `stream` SDK 도우미와 `EventHandler` 클래스를 사용하여
# Run을 생성하고 응답을 스트리밍합니다.

### 방법 2 - polling 방식 
RUN은 비동기식입니다. 즉, 터미널 상태에 도달할 때까지 Run 개체를 폴링하여 상태를 모니터링해야 한다는 뜻입니다. 'create_and_poll' SDK는 Run 생성과 완료 폴링을 모두 지원합니다.

In [ ]:
# `create_and_poll` 메서드를 사용하여 새로운 실행(run)을 생성하고 해당 실행이 완료될 때까지 폴링합니다.

In [ ]:
# 실행 상태 확인

## 코드 해석기를 이용한 파일 내용 분석
Assistant 에 전달된 파일은 이 Assistant를 사용하는 모든 Run에서 액세스할 수 있습니다.  

다음은 OpenAI의 code interpreter 도구를 사용하여 업로드된 mydata.csv 파일을 바탕으로 사용자에게 응답하는 샘플 코드입니다. 이 코드는 사용자가 특정 질문을 할 때 CSV 파일을 읽고, 해당 질문에 맞는 응답을 제공하는 예제입니다.

가정: 사용자가 학생의 이름을 입력하면 해당 학생의 나이와 성적을 반환하는 예제

#### 1. CSV 파일 업로드 및 어시스턴트 생성 코드

In [ ]:
# 예시 데이터 생성
# 데이터프레임 생성
# CSV 파일로 저장

In [ ]:
# "mydata.csv" 파일을 업로드
# 파일 ID를 사용하여 어시스턴트 생성

#### 2. 사용자 질문에 응답하는 코드 인터프리터 코드

In [ ]:
# 그런 다음, `stream` SDK 도우미와 `EventHandler` 클래스를 사용하여
# Run을 생성하고 응답을 스트리밍합니다.